In [15]:
%%capture
pip install -r requirements.txt

In [ ]:
# Import Necessary Modules
import pandas as pd  # For handling data in DataFrames
import numpy as np  # For numerical operations
from datetime import datetime, timedelta  # To handle dates and time manipulation
import json  # For handling JSON data
import sqlite3  # For interacting with SQLite databases
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns  # For advanced visualizations
from decimal import Decimal  # To handle precise decimal operations
%matplotlib inline  # Ensures that plots are displayed inline in Jupyter notebooks

# Initialize Database and Tables
def initialize_database():
    with sqlite3.connect('bank_simulation.db') as conn:
        cursor = conn.cursor()
        # Create the clients table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS clients (
                client_id INTEGER PRIMARY KEY,  # Unique ID for each client
                name TEXT,  # Client's name
                account_type TEXT,  # Type of account (savings, checking, etc.)
                currency TEXT,  # Account currency (USD, EUR, etc.)
                balance REAL  # Account balance
            )
        ''')
        # Create the ledger_entries table for double-entry bookkeeping
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS ledger_entries (
                entry_id INTEGER PRIMARY KEY AUTOINCREMENT,  # Auto-incremented entry ID
                date TEXT,  # Date of the entry
                transaction_id INTEGER,  # ID of the associated transaction
                account TEXT,  # Account name (e.g., Nostro Account, Client Deposits)
                debit REAL,  # Debit amount
                credit REAL,  # Credit amount
                description TEXT,  # Description of the entry
                FOREIGN KEY(transaction_id) REFERENCES transactions(transaction_id)  # Foreign key to link with transactions table
            )
        ''')
        # Create the transactions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS transactions (
                transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,  # Auto-incremented transaction ID
                date TEXT,  # Date of the transaction
                client_id INTEGER,  # ID of the client involved
                type TEXT,  # Type of transaction (deposit, withdrawal, etc.)
                amount REAL,  # Amount involved in the transaction
                currency TEXT,  # Currency of the transaction
                description TEXT,  # Description of the transaction
                FOREIGN KEY(client_id) REFERENCES clients(client_id)  # Foreign key linking to the clients table
            )
        ''')
        conn.commit()  # Commit the changes to the database

# Call the function to initialize the database and create the tables
initialize_database()

# Create Mock Clients to Populate the Clients Table
def insert_clients():
    # Sample client data to insert into the clients table
    clients = [
        (1, 'Alice Rossi', 'savings', 'EUR', 10000.00),
        (2, 'Bob Verdi', 'checking', 'USD', 5000.00),
        (3, 'Carla Bianchi', 'corporate', 'EUR', 25000.00),
        (4, 'David Nero', 'savings', 'GBP', 8000.00),
        (5, 'Eva Blu', 'checking', 'EUR', 12000.00),
        # More clients can be added here if needed
    ]
    with sqlite3.connect('bank_simulation.db') as conn:
        cursor = conn.cursor()
        # Insert or replace clients in the clients table
        cursor.executemany('''
            INSERT OR REPLACE INTO clients (client_id, name, account_type, currency, balance)
            VALUES (?, ?, ?, ?, ?)
        ''', clients)
        conn.commit()  # Commit the changes to the database

# Call the function to insert clients into the database
insert_clients()

# Function to Record Ledger Entries (for double-entry bookkeeping)
def record_ledger_entry(conn, date, transaction_id, account, debit, credit, description):
    cursor = conn.cursor()
    # Insert a new entry into the ledger_entries table
    cursor.execute('''
        INSERT INTO ledger_entries (date, transaction_id, account, debit, credit, description)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (date, transaction_id, account, debit, credit, description))

# Function to Handle Deposits
def deposit(client_id, amount, currency, date, description):
    with sqlite3.connect('bank_simulation.db') as conn:
        cursor = conn.cursor()
        # Update client's balance (increase in liabilities)
        cursor.execute('''
            UPDATE clients SET balance = balance + ?
            WHERE client_id = ? AND currency = ?
        ''', (amount, client_id, currency))
        # Record the transaction in the transactions table
        cursor.execute('''
            INSERT INTO transactions (date, client_id, type, amount, currency, description)
            VALUES (?, ?, 'deposit', ?, ?, ?)
        ''', (date, client_id, amount, currency, description))
        transaction_id = cursor.lastrowid  # Get the last inserted transaction ID
        # Double-entry ledger entries
        record_ledger_entry(conn, date, transaction_id, f'Nostro Account ({currency})', amount, 0, f'Deposit from Client {client_id}')
        record_ledger_entry(conn, date, transaction_id, f'Client Deposits ({currency})', 0, amount, f'Deposit to Client {client_id}')
        conn.commit()  # Commit the changes

# Function to Handle Withdrawals
def withdraw(client_id, amount, currency, date, description):
    with sqlite3.connect('bank_simulation.db') as conn:
        cursor = conn.cursor()
        # Update client's balance (decrease in liabilities)
        cursor.execute('''
            UPDATE clients SET balance = balance - ?
            WHERE client_id = ? AND currency = ?
        ''', (amount, client_id, currency))
        # Record the withdrawal in the transactions table
        cursor.execute('''
            INSERT INTO transactions (date, client_id, type, amount, currency, description)
            VALUES (?, ?, 'withdrawal', ?, ?, ?)
        ''', (date, client_id, -amount, currency, description))
        transaction_id = cursor.lastrowid  # Get the last inserted transaction ID
        # Double-entry ledger entries
        record_ledger_entry(conn, date, transaction_id, f'Client Deposits ({currency})', amount, 0, f'Withdrawal from Client {client_id}')
        record_ledger_entry(conn, date, transaction_id, f'Nostro Account ({currency})', 0, amount, f'Cash paid to Client {client_id}')
        conn.commit()  # Commit the changes

# Function to Simulate a Day of Transactions for All Clients
def simulate_one_day(date):
    with sqlite3.connect('bank_simulation.db') as conn:
        cursor = conn.cursor()
        # Retrieve all clients' IDs and their currencies
        cursor.execute('SELECT client_id, currency FROM clients')
        clients_data = cursor.fetchall()  # Get the results as a list of tuples
    for client_id, client_currency in clients_data:
        # Randomly choose a transaction type for the client
        txn_type = np.random.choice(['deposit', 'withdrawal', 'fee', 'loan_payment', 'fx_transaction', 'loan_disbursement'])
        amount = round(np.random.uniform(100, 1000), 2)  # Generate a random transaction amount
        description = f'Automatic {txn_type} transaction'  # Generate a transaction description
        # Perform the chosen transaction for the client
        if txn_type == 'deposit':
            deposit(client_id, amount, client_currency, date, description)
        elif txn_type == 'withdrawal':
            withdraw(client_id, amount, client_currency, date, description)
        # More transaction types such as loan payment, fx transaction can be handled similarly

# Function to Generate a Daily Ledger for Auditing
def generate_daily_ledger(date):
    with sqlite3.connect('bank_simulation.db') as conn:
        # Query all ledger entries for the given date
        df = pd.read_sql_query('''
            SELECT * FROM ledger_entries WHERE date = ?
        ''', conn, params=(date,))
    df.to_csv(f'daily_ledger_{date}.csv', index=False)  # Save the daily ledger to a CSV file
    total_debits = df['debit'].sum()  # Sum up the debits
    total_credits = df['credit'].sum()  # Sum up the credits
    print(f'Daily ledger for {date} generated with {len(df)} entries.')  # Print the number of ledger entries
    print(f'Total Debits: {total_debits}, Total Credits: {total_credits}')  # Print total debits and credits
    # Check if the ledger is balanced (i.e., total debits equal total credits)
    if round(total_debits, 2) == round(total_credits, 2):
        print('The ledger is balanced.')
    else:
        print('The ledger is NOT balanced.')

# Function to Simulate One Month of Transactions
def simulate_one_month():
    start_date = datetime.now() - timedelta(days=29)  # Start 30 days ago
    for i in range(30):  # Loop through each day in the month
        current_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')  # Get the current date
        simulate_one_day(current_date)  # Simulate transactions for the day
        generate_daily_ledger(current_date)  # Generate the daily ledger for the day

# Call the function to simulate one month of transactions
simulate_one_month()

# Generate Monthly Financial Statements According to IFRS Standards
def generate_monthly_reports():
    with sqlite3.connect('bank_simulation.db') as conn:
        # Query all ledger entries for the month
        df_ledger = pd.read_sql_query('SELECT * FROM ledger_entries', conn)
    # Calculate total debits and credits for the month
    total_debits = df_ledger['debit'].sum()
    total_credits = df_ledger['credit'].sum()
    print(f'Monthly Total Debits: {total_debits}, Total Credits: {total_credits}')
    # Verify if the ledger balances for the month
    if round(total_debits, 2) == round(total_credits, 2):
        print('The monthly ledger is balanced.')
    else:
        print('The monthly ledger is NOT balanced.')

    # Prepare the balance sheet (Assets, Liabilities, Equity)
    # Example calculations for Assets and Liabilities
    assets = {
        'Cash': df_ledger[df_ledger['account'].str.contains('Nostro Account')]['debit'].sum()
    }
    liabilities = {
        'Client Deposits': df_ledger[df_ledger['account'].str.contains('Client Deposits')]['credit'].sum()
    }
    # Display the balance sheet
    balance_sheet = pd.DataFrame({
        'Assets': pd.Series(assets),
        'Liabilities': pd.Series(liabilities)
    }).fillna(0)
    display(balance_sheet)

# Call the function to generate monthly financial statements
generate_monthly_reports()

# Visualize the Data (Transaction Distribution, Client Balances)
with sqlite3.connect('bank_simulation.db') as conn:
    df_transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
    df_clients = pd.read_sql_query('SELECT * FROM clients', conn)

# Plot the distribution of transaction types
plt.figure(figsize=(10,6))
sns.countplot(data=df_transactions, x='type')
plt.title('Transaction Types Distribution')
plt.xticks(rotation=45)
plt.show()

# Plot the balances of clients
plt.figure(figsize=(10,6))
sns.barplot(data=df_clients, x='name', y='balance', hue='currency')
plt.title('Client Balances by Currency')
plt.xticks(rotation=45)
plt.show()
